---
# Cairo University Faculty of Engineering
## Deep Learning
## Assignment 3

---

Please write your full name here
- **Name** : "-----------"

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# Autoencoders

## 1.1 Autoencoders Latent Space

The beauty of autoencoders is the possibility to see these internal representations. The bottleneck layer enforces data compression by having fewer units than input and output layers. Further limiting this layer to two or three units enables us to see how the autoencoder is organizing the data internally in two or three-dimensional latent space.

### Dataset

We will use the MNIST numbers dataset but load it using a different method
- Use tfds (tensorflow_datasets) to load the mnist numbers dataset
- In the tfds.load function:
  - Split the data into test and train
  - Shuffle the data
  - Make sure to load the labels with the data ('as_supervised' flag)


In [ ]:
# Your code here
(ds_train, ds_test_) = ###


**For ds_train**

- Fill in the missing code in 'preprocess function' to:
  - Cast the image to tf.float32
  - Normalize the image by dividing it by 255.0
- Batch the dataset
- Apply preprocess function to ds_train using 'map'
- cache and prefetch ds_train  (use tf.data.AUTOTONE)


**Question**

What is caching and prefetching? And why should we use them?

ANSWER

In [ ]:
batch_size = 256


def preprocess(image, label):
    # cast image to tf.float32
    image =
    # normalize image
    image =

    return image, image

# batch dataset
ds_train =
# apply preprocess using map
ds_train =
# cache data
ds_train =
# prefetch data
ds_train =

**For ds_test**

- Apply the same previous transformations to ds_test

In [ ]:
## Your Code here
# batch dataset
ds_test =
# apply preprocess using map
ds_test =
# cache data
ds_test =
# prefetch data
ds_test =

- Fill the missing code for the following function to return image and label for testing

In [ ]:
# return label for testing
def preprocess_with_label(image, label):
  #cast image to tf.float32
  image =
  # Normalize image by dividing by 255.0
  image =
  # return image and label
  return

# Use map to apply the above function to ds_test_ then batch (size=500), cache and prefectch it
ds_test_label =

### Latent Space Visualization

The plotting functions help visualize the encoding of inputs from high dimension into 2D latent space, and decoding back to the original dimension. The functions produces two plots:

- **Encoder map** shows the mapping from input images to coordinates (z1,z2) in latent space, with digit labels displayed in colorbar

- **Decoder** **grid** shows reconstructions from a grid of latent space coordinates (z1,z2)

![Latent space visualization](https://github.com/mpbrigham/colaboratory-figures/raw/master/nma/autoencoders/latent_space_plots_noaxis.png)

Build **TWO** fully connected models:
1. Encoder: (1 layer only)
  - layer 0 reshape input image to a vector
  - layer 1: 64 neurons
  - layer 2: 32 neurons
  - layer 3: 16 neurons
  - layer 4/encoding_dim : 2 neurons (no activation)
  - layer 5: Batch Normalization [What?](https://towardsdatascience.com/batch-normalization-in-3-levels-of-understanding-14c2da90a338)
2. Decoder: (1 layer only)
  - layer 0 : 16 neurons
  - layer 1 : 32 neurons
  - layer 2 : 64 neurons
  - layer 3 : output image vector shape (sigmoid)
  - layer 4: reshape vector to a 2D image

- All layers use ReLu activation unless stated otherwise.

**Question**

What is batch normalization? Why should  we use it?

ANSWER

In [ ]:
encoder =

decoder =

autoencoder = tf.keras.Sequential([encoder, decoder])

Compile the model using adam optimizer and binary cross entropy error loss function

- Train the network for `epochs=60` epochs and `validation_data=ds_test`, and visualize a few reconstructed samples.
    - Load the weights in ./checkpoints/autoencoder to your model as initial weights BEFORE training

In [ ]:
#### Your Code Here
# load the weights in ./checkpoints/autoencoder to ur built model

# train model
history=

- Load a batch of images from test set (ds_test) and display 10 of them in a SINGLE figure:
  - Display the original images in 1 row
  - Display their reconstruction from autoencoder in 2nd row

In [ ]:
## Your Code Here

We can use autoencoder to generate images!! Now that we have a trained autoencoder, we can ignore the encoder and use only the
decoder to sample from the latent variables to generate images. As we
did not use any activation in the last layer before the latent variables, the latent space is
unbounded and can be any real floating numbers, and there are hundreds of them!

Let's look at the latent space.
- Load a batch of images from test data with their labels (ds_test_label)
- Use the encoder model to predict their latent space dimensions

The color bar on the right indicates the intensity of
the digit labels.

In [ ]:
## Your code here

images, labels =
z_dims =


plt.figure(figsize=(8,8))
plt.scatter(z_dims[:,0], z_dims[:,1], c=labels, cmap='RdYlBu', s=3)
plt.colorbar()

The classes are not distributed uniformly. You can see clusters that are well separated from other classes

You might be able to see the non-uniformity better in the following images, which we will
generate by sweeping the latent variables from start of above axis to its end with a 1.0 interval

- z_samples: Create a 2D numpy array of shape (100, 2) that contains [z1, z2] values in the range (-5, 5) with a 1.0 interval
- images: Use the decoder model to construct images from those embeddings (z_samples)

In [ ]:
## Your code here
z__samples=

images =


grid_col = 10
grid_row = 10

f, axarr = plt.subplots(grid_row, grid_col, figsize=(grid_col, grid_row))

i = 0
for row in range(grid_row):
    for col in range(grid_col):
        axarr[row,col].imshow(images[i,:,:], cmap='gray')
        axarr[row,col].axis('off')
        i += 1
f.tight_layout(0.1, h_pad=0.2, w_pad=0.1)
plt.show()

This widget allows you to slide the latent variable
bars to generate images interactively. Have fun!

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
@interact
def explore_latent_variable(z1 = (-5,5,0.1),
                            z2 = (-5,5,0.1)):
    z_samples = [[z1, z2]]
    images = decoder.predict(z_samples)
    plt.figure(figsize=(2,2))
    plt.imshow(images[0,:,:], cmap='gray')

We can see that some digits are well represented in the sample distribution and they
are nicely drawn too. It is not the case for some other digits, which are blurry, and some
digits are even missing from the samples. The latter shows the shortcoming where there is
very little variation in generated images for those classes.

## 1.2 Anomaly Detection

### Dataset

In this example, you will train an autoencoder to detect anomalies on the ECG5000 dataset. This dataset contains 5,000 Electrocardiograms, each with 140 data points. You will use a simplified version of the dataset, where each example has been labeled either 0 (corresponding to an abnormal rhythm), or 1 (corresponding to a normal rhythm). You are interested in identifying the abnormal rhythms.

Note: This is a labeled dataset, so you could phrase this as a supervised learning problem. The goal of this example is to illustrate **anomaly detection** concepts you can apply to larger datasets, where you do not have labels available (for example, if you had many thousands of normal rhythms, and only a small number of abnormal rhythms).

How will you detect anomalies using an autoencoder? Recall that an autoencoder is trained to minimize reconstruction error. *You will train an autoencoder on the normal rhythms only, then use it to reconstruct all the data. Our hypothesis is that the abnormal rhythms will have higher reconstruction error. You will then classify a rhythm as an anomaly if the reconstruction error surpasses a fixed threshold.*

The dataset you will use is based on one from [timeseriesclassification.com](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000).


In [ ]:
# Download the dataset
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

The last column contains the labels. The other data points are the electrocadriogram data

- Read the labels and ECG data into different variables
- Use sklearn train_test_split to split the data into **20%** test and **80%** train data
- Use random state = 21

In [ ]:
#### Your Code Here #####
labels = ######
data = #####
train_data, test_data, train_labels, test_labels = #########

Normalize the data to [0,1] using min max normaliztion
- you might need to cast data to tf.float32
- avoid data leak

In [ ]:
######## Your Code Here #######
min_val =
max_val =

train_data =
test_data =

You will train the autoencoder using only the normal rhythms, which are labeled in this dataset as 1.
- Separate the normal rhythms data row from the abnormal rhythms into different variables.
- Do this for both test and train data

In [ ]:
######### Your Code Here ##########
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data =
normal_test_data =

anomalous_train_data =
anomalous_test_data =

Plot a normal ECG.

In [ ]:
plt.grid()
plt.plot(np.arange(140), normal_train_data[0])
plt.title("A Normal ECG")
plt.show()

Plot an anomalous ECG.

In [ ]:
plt.grid()
plt.plot(np.arange(140), anomalous_train_data[0])
plt.title("An Anomalous ECG")
plt.show()

### Model

Build **TWO** fully connected models:
1. Encoder:
  - layer 1 : 32 neurons
  - layer 2 : 16 neurons
  - layer 3 : 8 neurons
2. Decoder:
  - layer 1: 16 neurons
  - layer 2: 32 neurons
  - layer 3: 140 neurons

- All layers use ReLu activation except last layer should have a sigmoid avtivation.

In [ ]:
#### Your Code Here
encoder=
decoder=
autoencoder=

Compile the model using adam optimizer and mean absolute error loss function

In [ ]:
# Your code here

- Train the autoencoder using only the normal ECGs data
- Use entire test data for validation
- Use a batch size = 512
- epochs = 20
- shuffe the data

In [ ]:
#### Your Code Here
history=

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

### Detect Anamolies

You will classify an ECG as anomalous if the reconstruction error is greater than **one standard deviation** from the normal training examples.

First, let's plot a normal ECG from the training set, the reconstruction after it's encoded and decoded by the autoencoder, and the reconstruction error.

In [ ]:
encoded_data = encoder(normal_test_data).numpy()
decoded_data = decoder(encoded_data).numpy()

plt.plot(normal_test_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], normal_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

Create a similar plot, this time for an anomalous test example.

In [ ]:
##### Your Code here

Calculate the **mean average error** for normal examples from the training set, then classify future examples as anomalous if the *reconstruction error is higher than one standard deviation from the training set*.

- Predict the ECG reconstructions for **normal ECG train data** only
- Calculate the mean absolute error between the predicted reconstructions and the true ECG data (use tf.keras.losses)

In [ ]:
##### Your Code Here
reconstructions =
train_loss =



plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

- Choose a threshold value that is one standard deviations above the mean of the train_loss you just calculated




In [ ]:
##### Your Code Here
mean =
std =
threshold=

print("Threshold: ", threshold)

Note: There are other strategies you could use to select a threshold value above which test examples should be classified as anomalous, the correct approach will depend on your dataset. You can learn more with the links at the end of this assignment.

If you examine the reconstruction error for the anomalous examples in the test set, you'll notice most have greater reconstruction error than the threshold. By varing the threshold, you can adjust the [precision](https://developers.google.com/machine-learning/glossary#precision) and [recall](https://developers.google.com/machine-learning/glossary#recall) of your classifier.

- Predict the ECG reconstructions for **abnormal ECG test data** only
- Calculate the mean absolute error between the predicted reconstructions and the true ECG data (use tf.keras.losses)

In [ ]:
### Your Code Here
reconstructions =
test_loss =


plt.hist(test_loss[None, :], bins=50)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()

Classify an ECG as an anomaly if the reconstruction error is greater than the threshold.
- Fill in the missing code for the predict function to do so

In [ ]:
def predict(model, data, threshold):
  '''
  input: the model
  data: test data (contains raw data)
  threshold for anomaly classification
  '''
  ## Use model to predict reconstructions for data
  reconstructions =
  ## Calculate mae loss between reconstructions and true data
  loss =
  ## not_nomaly should contain 1 if loss < threshold and 0 otherwise
  not_anomaly =


  return not_anomaly

In [ ]:
def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))

Run the following cell to look at the classification acccuracy of your anomaly detection model

In [ ]:
preds = predict(autoencoder, test_data, threshold)
print_stats(preds, test_labels)

# Extra Sources
- Anomaly Detection
  - To learn more about anomaly detection with autoencoders, look at [interactive example](https://anomagram.fastforwardlabs.com/#/) built with TensorFlow.js by Victor Dibia.
  - For a real-world use case, you can learn how [Airbus Detects Anomalies in ISS Telemetry Data](https://blog.tensorflow.org/2020/04/how-airbus-detects-anomalies-iss-telemetry-data-tfx.html) using TensorFlow.